## Agentic AI based Travel Planning Assistant

# Task
Build an AI agent for trip itinerary creation using provided JSON data for flights, hotel suggestions, and places of interest. The agent should be able to generate a sample itinerary and summarize its capabilities, highlighting potential next steps.

## Load Datasets

### Subtask:
Load the provided JSON datasets for flights, hotel suggestions, and places of interest into suitable data structures (e.g., pandas DataFrames) for easy access and manipulation. You will need to provide the actual JSON data or file paths.


**Reasoning**:
First, I'll import the pandas library to enable data manipulation and DataFrame creation. Then, I will create placeholder JSON data for flights, hotels, and places of interest. This will allow us to demonstrate the loading process into pandas DataFrames as requested by the subtask.



In [1]:
import pandas as pd
import json
import requests
from langchain.tools import tool

In [2]:
from google.colab import userdata
import os

key = userdata.get("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = key

print("Key loaded:", os.getenv("OPENAI_API_KEY") is not None)

Key loaded: True


In [3]:
flights_df = pd.read_json("/content/flights.json")
hotels_df = pd.read_json("/content/hotels.json")
places_df = pd.read_json("/content/places.json")

print("Flights DataFrame head:")
print(flights_df.head())
print("\nHoyels DataFrame head:")
print(hotels_df.head())
print("\nPlaces DataFrame head:")
print(places_df.head())

Flights DataFrame head:
  flight_id    airline       from         to      departure_time  \
0    FL0001     IndiGo  Hyderabad      Delhi 2025-01-04 11:32:00   
1    FL0002  Air India      Delhi    Kolkata 2025-11-26 05:34:00   
2    FL0003   SpiceJet    Chennai  Hyderabad 2025-06-03 00:26:00   
3    FL0004  Air India  Bangalore     Mumbai 2025-05-13 13:18:00   
4    FL0005  Air India    Chennai  Bangalore 2025-02-08 03:08:00   

         arrival_time  price  
0 2025-01-04 15:32:00   2907  
1 2025-11-26 09:34:00   3779  
2 2025-06-03 01:26:00   5473  
3 2025-05-13 17:18:00   4764  
4 2025-02-08 05:08:00   3695  

Hoyels DataFrame head:
  hotel_id                name   city  stars  price_per_night  \
0  HOT0001  Grand Palace Hotel  Delhi      4             3897   
1  HOT0002      Comfort Suites  Delhi      5             3650   
2  HOT0003   Green Leaf Resort  Delhi      4             6123   
3  HOT0004       Sunrise Hotel  Delhi      2             4783   
4  HOT0005  Blue Lagoon Resort  

In [4]:
pip install langchain langchain-openai openai streamlit requests python-dotenv

Flight Search Tool

In [5]:
@tool
def search_flights(source: str, destination: str):
    """
    Finds the cheapest flight between two cities.
    """
    with open("/content/flights.json") as f:
        flights = json.load(f)

    results = [
        f for f in flights
        if f["source"].lower() == source.lower()
        and f["destination"].lower() == destination.lower()
    ]

    if not results:
        return "No flights found."

    cheapest = min(results, key=lambda x: x["price"])
    return cheapest

Hotel Search Tool

In [6]:
@tool
def recommend_hotel(city: str, max_price: int = 5000):
    """
    Recommends the best hotel in a city based on rating and budget.
    """
    with open("/content/hotels.json") as f:
        hotels = json.load(f)

    filtered = [
        h for h in hotels
        if h["city"].lower() == city.lower()
        and h["price_per_night"] <= max_price
    ]

    if not filtered:
        return "No hotels found within budget."

    best_hotel = max(filtered, key=lambda x: x["rating"])
    return best_hotel

Place Search Tool

In [7]:
@tool
def recommend_places(city: str):
    """
    Returns top tourist attractions in a given city.
    """
    with open("/content/places.json") as f:
        places = json.load(f)

    city_places = [
        p for p in places
        if p["city"].lower() == city.lower()
    ]

    if not city_places:
        return "No attractions found."

    top_places = sorted(
        city_places,
        key=lambda x: x["rating"],
        reverse=True
    )[:5]

    return top_places

Weather Tool

In [8]:
@tool
def get_weather(latitude: float, longitude: float):
    """
    Fetches weather forecast using Open-Meteo API.
    """
    url = (
        "https://api.open-meteo.com/v1/forecast"
        f"?latitude={latitude}&longitude={longitude}"
        "&daily=temperature_2m_max,weathercode"
        "&timezone=auto"
    )

    response = requests.get(url, timeout=10)
    response.raise_for_status()

    data = response.json()
    return data["daily"]

Budget Tool

In [9]:
@tool
def estimate_budget(
    flight_price: int,
    hotel_price_per_night: int,
    days: int
):
    """
    Estimates total trip budget.
    """
    hotel_cost = hotel_price_per_night * days
    local_expenses = 800 * days  # average food + travel

    total_cost = flight_price + hotel_cost + local_expenses

    return {
        "flight_cost": flight_price,
        "hotel_cost": hotel_cost,
        "local_expenses": local_expenses,
        "total_cost": total_cost
    }

AI Agent

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

tools = [
    search_flights,
    recommend_hotel,
    recommend_places,
    get_weather,
    estimate_budget
]

llm_with_tools = llm

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AGENTIC travel planning assistant. "
            "Generate a DAY-WISE itinerary (Day 1, Day 2, Day 3) "
            "including:\n"
            "- Selected flight\n"
            "- Selected hotel\n"
            "- Places to visit each day\n"
            "- Weather summary\n"
            "- Budget breakdown\n\n"
            "IMPORTANT:\n"
            "After selecting the flight and hotel, explicitly explain:\n"
            "• Why this flight was chosen\n"
            "• Why this hotel was chosen\n\n"
            "The explanations should be short, logical, and based on "
            "price, rating, convenience, and suitability for a budget trip."
        ),
        ("human", "{input}")
    ]
)

agent = (
    {"input": RunnablePassthrough()}
    | prompt
    | llm_with_tools
)


In [11]:
response = agent.invoke(
    "Plan a 3 day budget trip from Delhi to Goa "
    "with flights, hotels, places, weather and budget."
)

print(response.content)

**Day 1:**

**Flight:** IndiGo Airlines - Delhi to Goa
- Departure: 8:00 AM
- Arrival: 10:30 AM

**Hotel:** Backpacker Panda - Calangute
- Budget-friendly
- Good ratings for cleanliness and location

**Places to Visit:**
1. Calangute Beach
2. Baga Beach
3. Fort Aguada
4. Tito's Lane for nightlife

**Weather Summary:** Sunny with temperatures around 30°C

**Budget Breakdown:**
- Flight: $100
- Hotel (2 nights): $60
- Meals and Transportation: $50
- Entrance Fees and Activities: $40
- Total: $250

**Explanation:**
- **Flight:** IndiGo Airlines was chosen for its affordable price and convenient morning departure.
- **Hotel:** Backpacker Panda in Calangute was selected for its budget-friendly rates and good reviews on cleanliness and location.

**Day 2:**

**Places to Visit:**
1. Anjuna Beach
2. Chapora Fort
3. Saturday Night Market

**Weather Summary:** Partly cloudy with temperatures around 29°C

**Budget Breakdown:**
- Meals and Transportation: $40
- Entrance Fees and Activities: $30
- 